In [ ]:
first_time = True
!ls

sample_data


In [ ]:
if first_time:
  from google.colab import drive
  drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
if first_time:
  !apt-get update
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
  !tar xf spark-3.0.0-bin-hadoop2.7.tgz

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [42.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.

In [ ]:
# Set up Spark
!pip install -q findspark
!pip install py4j

!export JAVA_HOME=$(/usr/lib/jvm/java-8-openjdk-amd64 -v 1.8)
! echo $JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init("spark-3.0.0-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

if first_time:
  !pip install koalas
  !pip install --upgrade PyArrow

import time
import databricks.koalas as ks
import numpy as np
import pandas as pd
from IPython.display import display # for displaying multiple  dataframes from a single cell
import seaborn as sns
import matplotlib.pyplot as plt
import math
from numpy.linalg import norm
import os
os.environ["PYSPARK_PYTHON"] = "python3"

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

%matplotlib inline
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("AvazuCTR") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

     |████████████████████████████████| 204kB 5.7MB/s 
/bin/bash: /usr/lib/jvm/java-8-openjdk-amd64: Is a directory

     |████████████████████████████████| 798kB 5.8MB/s 
     |████████████████████████████████| 17.7MB 467kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


## load data

In [ ]:
!cp /content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/Data/ARCHIVED/train.csv .

In [ ]:
df = spark.read.csv("train.csv",inferSchema=True,header=True)

In [ ]:
df.show(10)

+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+
|                  id|click|    hour|  C1|banner_pos| site_id|site_domain|site_category|  app_id|app_domain|app_category|device_id|device_ip|device_model|device_type|device_conn_type|  C14|C15|C16| C17|C18|C19|   C20|C21|
+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+
| 1000009418151094273|    0|14102100|1005|         0|1fbe01fe|   f3845767|     28905ebd|ecad2386|  7801e8d9|    07d7df22| a99f214a| ddd2926e|    44956a24|          1|               2|15706|320| 50|1722|  0| 35|    -1| 79|
|10000169349117863715|    0|14102100|1005|         0|1fbe01fe|   f3845767|     28905ebd|ecad2386|  7801e8d9|    

## count

In [ ]:
df.count()

40428967

## Schema

In [ ]:
df.printSchema()

root
 |-- id: decimal(20,0) (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)



# Save data in parquet

Parquet has some advantages over csv, primarily ([source](https://www.youtube.com/results?search_query=spark++Parquet)):
- defined schemas
- predicate push down, e.g., filtering before processing

In [ ]:
'''
df.coalesce(4) #Returns a new DataFrame that has exactly numPartitions partitions. 
  .write #write a DataFrame to external storage systems (e.g. file systems, key-value stores, etc). Use DataFrame.write to access this.
  .mode("overwrite") #Specifies the behavior when data or table already exists.
  .parquet("parquet/filtered_train.csv")# Parquet storage formats is more efficient/fast with big data
# speed up sql querel by overcoming two major bottlenecks of distributed analytics: communication costs (IO bound) and data decoding (CPU bound).
'''
df.coalesce(4).write.mode('overwrite').parquet("/content/drive/MyDrive/LAI_OFFER/BIG_DATA/AvazuCTR/Data/parquet/train.csv")

coalesce is like repartition with minimal shuffling and data moval 

Complementary reading:
- coalesce is like repartition with minimal shuffling and data moval 
- Spark can run 1 concurrent task for every partition of an RDD (up to the number of cores in the cluster). If you're cluster has 20 cores, you should have at least 20 partitions (in practice 2–3x times more ) [ref](https://www.google.com/search?q=spark+number+of+partitions&oq=spark+number+of+partitions&aqs=chrome..69i57j0i7i30l7.9744j0j7&sourceid=chrome&ie=UTF-8)
- [partition-worker node,   excutor-task](http://beginnershadoop.com/2019/09/30/distribution-of-executors-cores-and-memory-for-a-spark-application/)
- [stackoverflow](https://stackoverflow.com/questions/24696777/what-is-the-relationship-between-workers-worker-instances-and-executors#:~:text=A%20node%20is%20a%20machine,many%20executors%2C%20for%20many%20applications.)

- refer to class 59
- excutor: a single JVM INSTANCE(/process?) ON A NODETHAT SERVERS A SINGLE SPARK application. An executor runs multiple tasks over its lifetime, and multiple tasks concurrently (?). A node may have several Spark executors and there are many nodes running Spark Executors for each client application.
- task: spark task represents a unit of work on a partition of a distributed dataset (RDD). 

An executor is a single JVM process which is launched for an application on a worker node. excutors are for running tasks, A single node can run multiple executors and executors for an application can span multiple worker nodes. An executor stays up for the duration of the Spark Application and runs the tasks in multiple threads. 
- core: A core is a basic computation unit of CPU and a CPU may have one or more cores to perform tasks at a given time. The more cores we have, the more work we can do. In spark, this controls the number of parallel tasks an executor can run. Liu: It seems cores can be shared by many excutors in the same single worker node.
- Workers hold many executors, for many applications. One application could have executors on many workers. In Colab, only a single application is lauched.